In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

import sys
sys.path.append('..')

from src.model import DeepCleanAutoencoder
from src.dataset import GWDataset
from src.utils import postprocess

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")

### Config

In [ ]:
MODEL_PATH = 'models/best_model.pth'
TEST_DATA_PATH = 'data/test_data.npz' 

# match training
SAMPLE_RATE = 2048 
BAND_MIN = 55.0
BAND_MAX = 65.0

INFERENCE_OVERLAP = 4.0

### Load Data and Model

In [ ]:
data = np.load(TEST_DATA_PATH)
strain_raw = data['strain']
witnesses_raw = data['witnesses']

print(f"Strain Shape: {strain_raw.shape}")
print(f"Witnesses Shape: {witnesses_raw.shape}")

test_dataset = GWDataset(
    strain_raw, 
    witnesses_raw, 
    SAMPLE_RATE, 
    overlap=INFERENCE_OVERLAP, 
    band_start=BAND_MIN, 
    band_end=BAND_MAX
)

model = DeepCleanAutoencoder(num_witnesses=witnesses_raw.shape[0]).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()
print("Model loaded")

### Run Inference + post process stuff

In [ ]:
segment_idx = 0

w_tensor, h_tensor_norm = test_dataset[segment_idx]

w_batch = w_tensor.unsqueeze(0).to(device)

with torch.no_grad():
    pred_normalized = model(w_batch)

noise_estimate = postprocess(
    pred_normalized, 
    test_dataset.strain_mean, 
    test_dataset.strain_std, 
    SAMPLE_RATE, 
    BAND_MIN, 
    BAND_MAX
)

start_idx = test_dataset.indices[segment_idx]
end_idx = start_idx + test_dataset.seg_len

segment_original = strain_raw[start_idx:end_idx]
segment_cleaned = segment_original - noise_estimate

print(f"Processed Segment {segment_idx} (Time: {start_idx/SAMPLE_RATE:.1f}s - {end_idx/SAMPLE_RATE:.1f}s)")

### Actual visualization stuff i dont feel like doing